In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
ostro_dir = 'Osteosarcoma-UT'



# Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [3]:
from sklearn.model_selection import train_test_split
import os

# Define the root directory containing subfolders for each class
root_dir = 'Osteosarcoma-UT'

# Get the list of class names
classes = os.listdir(root_dir)
print(classes)

# Split the data into training and validation sets
train_data = []
val_data = []
for cls in classes:
    cls_dir = os.path.join(root_dir, cls)
    images = [os.path.join(cls_dir, img) for img in os.listdir(cls_dir)]
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    train_data.extend([(img, cls) for img in train_images])
    val_data.extend([(img, cls) for img in val_images])

# Further split the training data into training and testing sets
train_images, test_images = train_test_split(train_data, test_size=0.2, random_state=42)

# Now you have train_images, val_data, and test_images containing paths to images for each set



['Viable', 'Non-Tumor', 'Non-Viable-Tumor']


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame with two columns: 'file_path' and 'label'
train_df = pd.DataFrame(train_data, columns=['file_path', 'label'])
val_df = pd.DataFrame(val_data, columns=['file_path', 'label'])
test_df = pd.DataFrame(test_images, columns=['file_path', 'label'])

# Split the data into file paths and labels
train_data = train_df['file_path']
train_labels = train_df['label']

val_data = val_df['file_path']
val_labels = val_df['label']

test_data = test_df['file_path']
test_labels = test_df['label']

# You can create separate data generators for each set
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 914 validated image filenames belonging to 3 classes.
Found 230 validated image filenames belonging to 3 classes.
Found 183 validated image filenames belonging to 3 classes.


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [6]:
from keras_unet.models import custom_unet
from keras.optimizers import Adam
from keras.metrics import MeanIoU

# Define the input shape (assuming 224x224 images)
input_shape = (224, 224, 3)

# Define the number of classes in your segmentation task
num_classes = 3  # Adjust this according to your task

# Load the pre-trained U-Net model
model = custom_unet(
    input_shape,
    filters=64,
    num_classes=num_classes,
    output_activation='softmax'  # Use softmax activation for multi-class segmentation
)

# Compile the model
model.compile(
    optimizer=Adam(), 
    loss='categorical_crossentropy',  # Use categorical cross-entropy for multi-class segmentation
    metrics=['accuracy', MeanIoU(num_classes=num_classes)]
)

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=100,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


2024-02-21 15:44:03.734415: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-21 15:44:03.734439: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-21 15:44:03.734446: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-21 15:44:03.734708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-21 15:44:03.735179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 64)         1728      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 224, 224, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 spatial_dropout2d (Spatial  (None, 224, 224, 64)         0         ['batch_normalization[0][0

ValueError: in user code:

    File "/Users/uvaishnav/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/uvaishnav/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/uvaishnav/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/uvaishnav/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/uvaishnav/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/uvaishnav/anaconda3/lib/python3.11/site-packages/keras/src/engine/compile

In [7]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

5/5 [==============================] - 3s 373ms/step - loss: 0.1911 - accuracy: 0.9250
Test accuracy: 0.925000011920929
